In [ ]:
import cv2
import matplotlib.pyplot as plt

# 解説2
# 画像を読み込む
img1 = cv2.imread("./images/car1.jpeg", cv2.IMREAD_GRAYSCALE)  # クエリ画像
img1 = cv2.resize(img1, dsize=None, fx=0.2, fy=0.2)
img2 = cv2.imread("./images/car2.jpeg", cv2.IMREAD_GRAYSCALE)  # クエリ画像
img2 = cv2.resize(img2, dsize=None, fx=0.2, fy=0.2)

# 解説3
# SIFT検出器を初期化する
sift = cv2.SIFT_create()

# SIFTを用いてキーポイントと記述子を見つける
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# デフォルトパラメータでBFMatcherを作成
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# レシオテストを適用
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

# 解説4
# cv2.drawMatchesKnnはマッチング結果としてリストのリストを期待する
img3 = cv2.drawMatchesKnn(
    img1,
    kp1,
    img2,
    kp2,
    good,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

plt.imshow(img3)

In [ ]:
img3 = cv2.drawMatchesKnn(
    img1,
    kp1,
    img2,
    kp2,
    good,
    None,
    # flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)
plt.imshow(img3)